데이터 : 해양수산부 국립수산과학원

### 데이터불러오기

In [26]:
import requests
import pandas as pd
import json

In [27]:
url = 'http://apis.data.go.kr/1520635/OceanBiospeciesInfoService/getOceanBiospeciesResult?serviceKey=dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==&numOfRows=1000&pageNo=1'
params ={'serviceKey' : 'dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==',
        'numOfRows' : '10', 
        'pageNo' : '1' }

response = requests.get(url=url)
print(url,params)

http://apis.data.go.kr/1520635/OceanBiospeciesInfoService/getOceanBiospeciesResult?serviceKey=dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==&numOfRows=1000&pageNo=1 {'serviceKey': 'dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==', 'numOfRows': '10', 'pageNo': '1'}


### xml->dataframe

In [28]:
_df = pd.read_xml(response.text, xpath='.//item')

C:\Users\daega\AppData\Local\Temp\ipykernel_31124\2446481714.py:1: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  _df = pd.read_xml(response.text, xpath='.//item')


### 전처리

행 삭제 (필요한것만 남기기)

In [29]:
df = _df[["mfClassKor","mfDisinfo", "mfDistribution", "mfFamilyKor", "mfFeature", "mfSpeciesKor"]]
df.head()

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor
0,조기어강,None,한국 남부 태평양과 인도양의 열대 및 온대 해역,돛새치과,좌우 새조골막은 하나로 융합되어 있으며 새파는 없다.양턱과 구개골에는 작지만 균일한...,청새치
1,조기어강,None,"북서태평양 (한국, 일본, 동중국해, 대만)",쥐치과,"몸은 타원형에 가까우며 매우 측편되어 있다. 체고는 높으며, 주둥이 끝은 뾰족하고...",쥐치
2,조기어강,None,"우리나라 남부, 일본 북해도 이남, 동중국해까지 분포한다.",쥐치과,몸은 타원형에 가까우며 옆으로 매우 납작하고 몸높이는 높다. 주둥이 끝은 뾰족하고 ...,쥐치
3,조기어강,None,None,쥐치과,None,쥐치
4,조기어강,전연안,"우리나라 전 연안, 일본 북해도이남, 동중국해, 대만",쥐치과,"몸은 타원형에 가깝고 측편되며, 체고가 높다. 주둥이는 뾰족하고 꼬리자루는 짧으며,...",쥐치


중복제거 & 결측치제거

In [32]:
df.drop_duplicates(subset='mfSpeciesKor',inplace=True)
df.dropna(subset='mfSpeciesKor',inplace=True)

C:\Users\daega\AppData\Local\Temp\ipykernel_31124\1596300023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='mfSpeciesKor',inplace=True)
C:\Users\daega\AppData\Local\Temp\ipykernel_31124\1596300023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset='mfSpeciesKor',inplace=True)


우리나라 안 사는 어류종 삭제

In [51]:
# 한국 관련 아닌 키워드를 추출하여 조건을 생성
condition = (
    (df['mfDisinfo'] != '우리나라') & 
    (df['mfDisinfo'] != '한국') & 
    (df['mfDisinfo'] != '전연안') & 
    (~df['mfDistribution'].str.contains("우리나라", na=False)) & 
    (~df['mfDistribution'].str.contains("한국", na=False)) &
    (~df['mfDistribution'].str.contains("동해", na=False)) &
    (~df['mfDistribution'].str.contains("서해", na=False)) & 
    (~df['mfDistribution'].str.contains("남해", na=False)) & 
    (~df['mfDistribution'].str.contains("제주", na=False)) & 
    (~df['mfDistribution'].str.contains("울릉", na=False)) &
    (~df['mfDistribution'].str.contains("독도", na=False))
)

# 조건에 맞는 행 삭제
df = df[~condition]

고래 & 상어 & 해파리 삭제

In [59]:
condition = (
    (df['mfSpeciesKor'].str.contains("해파리", na=False)) |
    (df['mfSpeciesKor'].str.contains("상어", na=False)) |
    (df['mfSpeciesKor'].str.contains("고래", na=False)) |
    (df['mfFamilyKor'].str.contains("해파리", na=False)) |
    (df['mfFamilyKor'].str.contains("상어", na=False)) |
    (df['mfFamilyKor'].str.contains("고래", na=False)) 
)

df = df[~condition]

지역별로 나누기

In [61]:
df['지역'] = None
df.head()

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor,지역
0,조기어강,None,한국 남부 태평양과 인도양의 열대 및 온대 해역,돛새치과,좌우 새조골막은 하나로 융합되어 있으며 새파는 없다.양턱과 구개골에는 작지만 균일한...,청새치,None
1,조기어강,None,"북서태평양 (한국, 일본, 동중국해, 대만)",쥐치과,"몸은 타원형에 가까우며 매우 측편되어 있다. 체고는 높으며, 주둥이 끝은 뾰족하고...",쥐치,None
7,이형세대강,"제주도,완도,독도","일본, 오스트레일리아, 인도양에 분포하고 우리나라에서는 제주도, 독도, 완도에 분포한다.",스포로크누스과,뿌리는 상당히 큰 원추상반이며 황갈색의 털로 덮였다. 털은 뿌리의 상당히 상부에까지...,스포로크누스,None
8,복족강,None,"한국, 일본(홋카이도, 혼슈, 시코쿠, 큐우슈우, 초시)",밤고둥과,"패각은 난원형, 나탑은 낮은 원추형, 나층은 7층, 각 층은 팽윤되어 있다.\r\r...",개울타리고둥,None
9,None,"주문진,속초,부산,비진도,거제도,욕지도,완도,여수,어청도,제주도","일본에 분포되고 우리나라에서는 주문진, 속초, 부산, 비진도, 거제도, 욕지도, 완...",모자반과,몸의 높이는 2∼3m 이며 대체로 전기한 종과 매우 비슷하다. 그리고 계절과 산지에...,꽈배기모자반,None


In [67]:
### 우선순위 : 서해-> 울릉 -> 제주 -> 부산 -> 동해 -> 남해
## 우선순위대로 하기 위해 반대로 코딩하면 됨
## 서해 어류종이 너무 적어 1순위 (마지막에 count)

ulleung = '울릉도'
jeju = '제주'
busan = '부산'
east = '동해'
south = '남해'
west = '서해'

In [90]:
# 남해

condition = (
    (df['mfDisinfo'].str.contains(south, na=False)) |
    (df['mfDistribution'].str.contains(south, na=False))
)

df.loc[condition, ['지역']] = south

print(south , len(df[df['지역']==south]))

남해 62


In [91]:
# 동해

condition = (
    (df['mfDisinfo'].str.contains(east, na=False)) |
    (df['mfDistribution'].str.contains(east, na=False))
)

df.loc[condition, ['지역']] = east

print(east , len(df[df['지역']==east]))

동해 66


In [92]:
# 부산

condition = (
    (df['mfDisinfo'].str.contains(busan, na=False)) |
    (df['mfDistribution'].str.contains(busan, na=False))
)

df.loc[condition, ['지역']] = busan

print(busan , len(df[df['지역']==busan]))

부산 52


In [94]:
# 제주도

condition = (
    (df['mfDisinfo'].str.contains(jeju, na=False)) |
    (df['mfDistribution'].str.contains(jeju, na=False))
)

df.loc[condition, ['지역']] = jeju

print(jeju , len(df[df['지역']==jeju]))

제주 64


In [95]:
# 울릉도

condition = (
    (df['mfDisinfo'].str.contains(ulleung, na=False)) |
    (df['mfDistribution'].str.contains(ulleung, na=False))
)

df.loc[condition, ['지역']] = ulleung

print(ulleung , len(df[df['지역']==ulleung]))

울릉도 24


In [97]:
# 서해

condition = (
    (df['mfDisinfo'].str.contains(west, na=False)) |
    (df['mfDistribution'].str.contains(west, na=False))
)

df.loc[condition, ['지역']] = west

print(west , len(df[df['지역']==west]))

서해 20


지역별 개수 확인

In [98]:
value_counts = df['지역'].value_counts()
print(value_counts)

지역
제주     53
동해     42
남해     25
울릉도    23
부산     21
서해     20
Name: count, dtype: int64


### 20개씩만 남기기

In [105]:
group = df.groupby('지역').head(20)
df = group.sort_values(by='지역')

df

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor,지역
504,조기어강,남해안,"우리나라 남해, 일본 중부이남, 동중국해, 대만, 필리핀",나비고기과,"몸은 원통형에 가깝고, 체고가 높고 매우 측편하며, 머리는 짧은 편이다. 주둥이는 ...",세동가리돔,남해
958,조기어강,서남해안,"우리나라 서·남해, 일본 북해도 이남, 발해, 황해, 동중국해",도미과,"몸은 타원형으로 측편하며, 주둥이는 약간 돌출한다. 양 턱의 앞쪽에는 각각 3쌍의 ...",감성돔,남해
927,조기어강,None,"우리나라의 남해안과 인접한 강 하구와 일본, 대만, 하와이 및 서인도양 연안에 분포한다.",망둑어과,"등지느러미 기조수는 5 11, 뒷지느러미 기조수는 11, 가슴지느러미 기조수는 19...",열동갈문절,남해
908,조기어강,None,"태평양 서부 (한국 남해, 일본 남부, 동중국해, 대만)",빨간양태과,등지느러미는 2개로 가까이 위치하지만 분리되어 있고 제1등지느러미의 3번째 가시가 ...,빨간양태,남해
887,조기어강,남해안,"우리나라 남해, 일본 남부해, 동중국해",양볼락과,"몸은 타원형으로 측편하고, 머리에 단단한 가시들이 발달되어 있다. 눈 아래쪽에 뒤쪽...",붉감펭,남해
...,...,...,...,...,...,...,...
292,연갑강,None,제주도를 포함한 우리나라 전 연안의 조간대에서부터 수심 100m정도까지의 저서 암반...,부채게과,"갑각은 부채꼴 모양이며 갑각의 등 면에 뚜렷한 주름들이 있으며, 작은 과립들이 있다...",꽃부채게,제주
267,조기어강,None,MORI(1952)는 마산과 진해에서 본 종의 국내 출현을 보고하는 KIM 등(19...,망둑어과,"제1등지느러미 극조수는 5, 제2등지느러미 기조수는8~9, 뒷지느러미 기조수는9, ...",날개망둑,제주
175,조기어강,None,"북서태평양(한국 제주도, 일본, 중국)",신월가자미과,체형은 타원형이고 눈은 몸의 오른쪽에 위치한다. 유안측의 가슴지느러미 길이가 무안측...,신월가자미,제주
260,녹조식물강,서귀포,"일본, 인도네시아, 폴리네시아, 아메리카 태평양안에 분포되고, 우리나라에는 제주도 ...",아나디오메네과,"몸은 크기가 3∼5cm 이고 꽃잎 모양으로 뭉쳐서 나고 부채꼴이며, 외견상 나뭇잎맥...",잎맥말,제주


### 저장